In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
trial = random_population()

In [2]:
df = pd.read_excel('tomiga5gene.xlsx')
ge_original = np.array(df)
n = len(ge_original[1])
n_of_paramters = 2*n*(n+1)
t = len(ge_original)
F = 0.02
CR = 0.5

In [3]:
Answer = np.array([[15,10,0. ,  0. ,  1. ,  0. , -0.1, 2. ,  0. ,  0. ,  0. ,  0.],
[10,10,2. ,  0. ,  0. ,  0. ,  0., 0. ,  2. ,  0. ,  0. ,  0.],
[10,10,0. , -0.1,  0. ,  0. ,  0., 0. , -0.1,  2. ,  0. ,  0.],
[8,10,0. ,  2. ,  0. ,  0. , -1.,0. ,  0. ,  0. ,  2. ,  0.],
[10,10, 0. ,  0. ,  0. ,  2. ,  0.,0. ,  0. ,  0. ,  0. ,  2.]])

In [4]:
def rand(l,h):
    return(np.round(np.random.randint(low=l, high=h),2))

In [5]:

def random_population():
    Parameters = []
    
    
    for i in range(n):
        temp = []
        for j in range(2):
            temp.append(rand(0,20))
        for k in range(int(n_of_paramters/n)-2):
            zerorand = np.random.randint(0,2)
            if(zerorand==0):
                temp.append(0)
            else:
                temp.append(rand(-3,3))
            
        Parameters.append(temp)
    
    #Zero = np.random.randint(low = 0,high= 2 ,size = (5,12))
    #Zero = np.where(Zero==2, 1, Zero)
    
    
    return(np.array(Parameters))       

In [6]:
def calc_dydt(alpha, beta, g, h, exp ):
    
    
    Generation = alpha * (np.prod(pow(exp,g)))
    Degeneration = beta* (np.prod(pow(exp,h)))
    
    if(np.isnan(Generation) or np.isnan(Degeneration)):
        return(0)
    
    return (Generation-Degeneration)
    
def calc_ge(p):
    
    
    original = ge_original.copy()
    ge_calc = [original[0]]
    H=0.01 #step size
    
    
    for i in range((t-1)*(5)):
     
        
        single_exp=[]
       
        for j in range(n):
            
            
            
            cur_p = p[j]
            
            Alpha = cur_p[0]
            Beta = cur_p[1]
            g = cur_p[2:n+2]
            h = cur_p[n+2:(2*(n+1))]
            
            
            
            ge_temp = ge_calc[i].copy()
            
            
            value = ge_calc[i][j].copy()
            
            
            
            k1 = calc_dydt(Alpha,Beta,g,h,ge_temp)
            
            if(k1==0):
                return(0)
            
            k2 = calc_dydt(Alpha,Beta,g,h,ge_temp+(H*0.5*k1))
            
            if(k2==0):
                return(0)
            
            k3 = calc_dydt(Alpha,Beta,g,h,ge_temp+(H*0.5*k2))
            
            if(k3==0):
                return(0)
            
            
            k4 = calc_dydt(Alpha,Beta,g,h,ge_temp+(k3*H))
            
            if(k4==0):
                return(0)
            
            next_step = value + ((1/6)*(k1+2*k2+2*k3+k4)*H)
            
            #next_step = value + (k1*H)

            #rangeKutta Method call
            
            
            
            
            
            single_exp.append(next_step)
            
        ge_calc.append(single_exp)
    
    ge_calc = np.array(ge_calc)
    ge=[]
    for x in range(len(ge_calc)):
        if(x%5==0):
            ge.append(ge_calc[x])
    ge = np.array(ge)
    
    
    return(ge)

In [7]:
def fitness(Calculated):
    Original = ge_original.copy()
    fit = ((Calculated-Original)/Original)**2
    return(np.sum((np.sum(fit, axis=0))))    

In [113]:
def mutation(cur,counter):
    
    
    #print("this counter",counter)
    if(counter%20==0.000000000):
        #print("i got here from counter",counter)
        return(trig_mutation())
    else:
        R1 = random_population()
        R2 = random_population()
        R3 = random_population()
        return(R1+F*(R2-R3))

def trig_mutation():
    
    R1 = best_values[len(best_values)-1][2].copy()
    F_R1 = best_values[len(best_values)-1][0].copy()
    
    R2 = best_values[len(best_values)-2][2].copy()
    F_R2 = best_values[len(best_values)-2][0].copy()
        
    R3 = best_values[len(best_values)-3][2].copy()
    F_R3 = best_values[len(best_values)-3][0].copy()
    
    #print(F_R1,F_R2,F_R3) 
    
    
    P = F_R1+F_R2+F_R3
    P1 = F_R1/P
    P2 = F_R2/P
    P3 = F_R3/P
    average = (R1+R2+R3)/3
    weighted1 = (P2-P1)*(R1-R2)
    weighted2 = (P3-P2)*(R2-R3)
    weighted3 = (P1-P3)*(R3-R1)
    return(average+weighted1+weighted2+weighted3)
    
    

In [ ]:
#next_generation(trial)

In [9]:
def crossover(mut,target):
    
    rand_j = np.random.uniform(low=0,high=1,size=target.shape)
    CR_array = np.full(target.shape,CR)
    random_index_i = np.random.randint(0,4)
    random_index_j = np.random.randint(0,11)
    compare = rand_j<=CR_array
    trial = (mut.copy()*compare)+(target.copy()*~compare)
    trial[random_index_i][random_index_j]=mut[random_index_i][random_index_j].copy()
    
    return(trial)   

In [10]:
def check_bounds(trial):
    
    low_bounds = np.array(np.concatenate((np.full((n,2),0), np.full((n,n),-3),np.full((n,n),-3)), axis=1))
    upper_bounds = np.array(np.concatenate((np.full((n,2),20), np.full((n,n),3),np.full((n,n),3)), axis=1))
    
    while( (trial<low_bounds).any() or (trial>upper_bounds).any()):
        trial[trial>upper_bounds]=2*(upper_bounds[trial>upper_bounds])-trial[trial>upper_bounds]
        trial[trial<low_bounds]=2*(low_bounds[trial<low_bounds])-trial[trial<low_bounds]

    
    return(trial)
    
    

In [16]:
def next_generation(current,count):
    #print("here the counter is :", count)
    mutated = mutation(current,count)
    
    return(check_bounds(crossover(mutated,current)))
    
    

In [130]:
count=101
all_values = []
best_values = []
g = random_population()        
best_value = 0
best_fitness = float("NaN")
best_generation = 0
valu = calc_ge(g)
if(type(value)  != type(0)):
    best_fitness = fitness(value)
print("starting of fitness with", best_fitness)
while(best_fitness>0.5 or np.isnan(best_fitness)):
    
    g = next_generation(g,count)
    #print(count)
    Calculated = calc_ge(g)
    
    while(type(Calculated)==type(0)):
        g = next_generation(g,count)
        Calculated = calc_ge(g)
        if(len(best_values)>3):
            
            count=count+1
    
    
    new_fitness = fitness(Calculated)
    if(new_fitness<best_fitness or np.isnan(best_fitness)):
        best_fitness=new_fitness.copy()
        best_value = Calculated.copy()
        best_generation = g.copy()
        best_values.append([best_fitness,best_value,best_generation])
        print("for loop ",i)
        print("best fitness: ",new_fitness)
        print("Similar Parameters",np.sum(best_generation.astype(int)==Answer))
    count=count+1
    all_values.append([new_fitness,Calculated,g])

total_time_taken= time.time() - start_time
print("==========================================================")    
print("loop has ended")
print("==========================================================")    
print("results:")
print("total time taken: ", total_time_taken/60, "mins")
print("best fitness: ",new_fitness)
print("Similar Parameters: ",np.sum(best_generation.astype(int)==Answer))
print("Best Generation:\n", best_generation)

starting of fitness with nan


<ipython-input-6-143305e48728>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-143305e48728>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


for loop  999
best fitness:  26961994.670447685
Similar Parameters 23
for loop  999
best fitness:  372814.87778336543
Similar Parameters 20


<ipython-input-6-143305e48728>:4: RuntimeWarning: overflow encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-143305e48728>:5: RuntimeWarning: overflow encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))
<ipython-input-6-143305e48728>:10: RuntimeWarning: invalid value encountered in double_scalars
  return (Generation-Degeneration)


for loop  999
best fitness:  4150.535313304767
Similar Parameters 25
for loop  999
best fitness:  722.1347473893251
Similar Parameters 29
for loop  999
best fitness:  60.51427819827947
Similar Parameters 22
for loop  999
best fitness:  53.72455410733854
Similar Parameters 32
for loop  999
best fitness:  51.78582063762667
Similar Parameters 23
for loop  999
best fitness:  18.38246172897413
Similar Parameters 30


...

<ipython-input-7-607ae45484b1>:3: RuntimeWarning: overflow encountered in square
  fit = ((Calculated-Original)/Original)**2


for loop  999
best fitness:  8.775311562081448
Similar Parameters 29
for loop  999
best fitness:  7.503419485168033
Similar Parameters 26
for loop  999
best fitness:  5.858106271029608
Similar Parameters 35
for loop  999
best fitness:  4.18045691099657
Similar Parameters 29
for loop  999
best fitness:  3.5933972122889903
Similar Parameters 33
for loop  999
best fitness:  2.5049735387539243
Similar Parameters 31


<ipython-input-6-143305e48728>:4: RuntimeWarning: invalid value encountered in double_scalars
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-143305e48728>:5: RuntimeWarning: overflow encountered in double_scalars
  Degeneration = beta* (np.prod(pow(exp,h)))
<ipython-input-6-143305e48728>:66: RuntimeWarning: overflow encountered in double_scalars
  next_step = value + ((1/6)*(k1+2*k2+2*k3+k4)*H)
<ipython-input-6-143305e48728>:66: RuntimeWarning: invalid value encountered in double_scalars
  next_step = value + ((1/6)*(k1+2*k2+2*k3+k4)*H)


for loop  999
best fitness:  2.1585893046984044
Similar Parameters 34


KeyboardInterrupt: 

In [127]:
719.7002181213387<float("NaN")

False

In [92]:
calc_ge(check_bounds(trig_mutation()))

96228610.54313815 9187935568.310848 1.1968468440022897e+22


<ipython-input-6-143305e48728>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))


0

In [101]:
fitness(calc_ge(check_bounds(trig_mutation())))

19.155305220100043 22.343747627632872 27.136352955514198


21.35948682171812

In [124]:
np.sum(best_generation.astype(int)==Answer)

24

In [123]:
best_generation

array([[11.08, 17.86, -1.04,  2.  ,  0.  ,  0.  , -0.04, -0.02,  0.  ,
         0.  ,  0.  , -0.06],
       [ 7.12,  7.98,  0.04, -3.  ,  0.02,  0.94,  0.  ,  1.  , -1.98,
         0.  , -3.  ,  0.08],
       [ 8.24, 18.16,  0.02, -0.02, -1.  ,  2.04,  0.94,  0.  ,  1.98,
         1.  ,  0.  ,  0.04],
       [ 0.26,  2.86, -0.04, -3.  , -1.  ,  0.  ,  1.98,  0.94,  1.98,
        -0.02,  0.04,  0.02],
       [ 7.1 ,  3.06, -0.04, -0.02,  1.92,  0.  , -1.  ,  1.04,  0.  ,
         0.  , -1.96,  0.04]])

In [122]:
best_generation.astype(int)

array([[11, 17, -1,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 7,  7,  0, -3,  0,  0,  0,  1, -1,  0, -3,  0],
       [ 8, 18,  0,  0, -1,  2,  0,  0,  1,  1,  0,  0],
       [ 0,  2,  0, -3, -1,  0,  1,  0,  1,  0,  0,  0],
       [ 7,  3,  0,  0,  1,  0, -1,  1,  0,  0, -1,  0]])